In [125]:
import json
import pandas as pd
import requests
import pickle

In [126]:
follows = pd.read_csv('../../shared-folder-gald/data/follow-link.csv')

In [127]:
# users = pd.read_csv('../../shared-folder-gald/data/user_info_ALL.p')

with open('../../shared-folder-gald/data/user_info_ALL.p', 'rb') as f:
    users = pickle.load(f)

In [128]:
prev_users = pd.read_csv('../data/liked_videos.csv')

In [129]:
usernames = list(users.keys())

In [130]:
# Get the set of usernames in prev_users
prev_usernames = set(prev_users['username'])

# Remove existing usernames from the full list
usernames = [user for user in usernames if user not in prev_usernames]

In [131]:
with open("../../shared-folder-gald/keys2.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [132]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [133]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [134]:
def get_liked_info(username, access_token,fields ="id, username" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/liked_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [135]:
access_token = get_access_token(client_key, client_secret)

Ran up until [3003:4004]

In [ ]:
liked_videos = []
for username in usernames[4004:5005]:
    liked_videos.append(get_liked_info(username, access_token))

In [ ]:
combined = {}
for i, username in enumerate(usernames[4004:5005]):
    combined[username] = liked_videos[i]

In [138]:
rows = []

for key, value in combined.items():
    if value and 'user_liked_videos' in value:  # Ensure key has data
        for video in value['user_liked_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'id': video['id'],  # Video ID
                'creator': video['username']  # Video username
            })

# Create DataFrame
likes_df = pd.DataFrame(rows)

# Display DataFrame
print(likes_df)

                 username                   id               creator
0              mscat2u517  7489983072725765406       booktokqutiepie
1              mscat2u517  7490063219403296031     elysiawagesauthor
2              mscat2u517  7489531769813781802       thebetterboxllc
3              mscat2u517  7490017982635117855            dandkbooks
4              mscat2u517  7489963891238292782        stacey.l.pagan
..                    ...                  ...                   ...
259  couponingwithchantal  7486867598123044126    tc.roots.creations
260  couponingwithchantal  7485844794791578911      savingwithshayna
261  couponingwithchantal  7486580978987650335       _alext_latonyat
262  couponingwithchantal  7485901028852911390    tc.roots.creations
263  couponingwithchantal  7485493928599358751  couponingwithchantal

[264 rows x 3 columns]


In [139]:
likes_df.to_csv("../../shared-folder-gald/data/liked_videos.csv", mode="a", index=False, header=False)

In [140]:
likes = pd.read_csv("../../shared-folder-gald/data/liked_videos.csv")

In [141]:
likes['username'].nunique()

1018